## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## NER 

Named Entity Recognition (NER) is a process in computational linguistics that involves identifying and classifying key elements in text into predefined categories. In this context, NER focuses on discerning terms relevant to heart disease risk factors and personal health information.

- **Model**: `en.med_ner.risk_factors.pipeline`
- **Model Description**: This pretrained pipeline is built on top of the ner_risk_factors model, designed specifically for identifying Heart Disease Risk Factors and Personal Health Information. The model recognizes entities including CAD, DIABETES, FAMILY_HIST, HYPERLIPIDEMIA, HYPERTENSION, MEDICATION, OBESE, PHI, and SMOKER.



In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "en-med-ner-risk-factors-pipeline"

content_type = "application/json"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

---------------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3


# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data_dicts):
    for data_dict in data_dicts:
        json_input_data = json.dumps(data_dict)
        i = 1
        input_file_name = f'inputs/real-time/input{i}.json'
        output_file_name = f'outputs/real-time/out{i}.out'

        while os.path.exists(input_file_name) or os.path.exists(output_file_name):
            i += 1
            input_file_name = f'inputs/real-time/input{i}.json'
            output_file_name = f'outputs/real-time/out{i}.out'

        os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
        os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

        with open(input_file_name, 'w') as f:
            f.write(json_input_data)

        s3_client.put_object(Bucket=s3_bucket, Key=f"{model_name}/validation-input-json/real-time/{os.path.basename(input_file_name)}", Body=bytes(json_input_data.encode('UTF-8')))

        response = sm_runtime.invoke_endpoint(
            EndpointName=model_name,
            ContentType=content_type,
            Accept="application/json",
            Body=json_input_data,
        )

        # Process response
        response_data = json.loads(response["Body"].read().decode("utf-8"))
        df = pd.DataFrame(response_data['predictions'])
        display(df)

        # Save response data to file
        with open(output_file_name, 'w') as f_out:
            json.dump(response_data, f_out, indent=4)


### Initial Setup

In [7]:
docs = [
    '''HISTORY OF PRESENT ILLNESS: The patient is a 40-year-old white male who presents with a chief complaint of "chest pain".

The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admits prior episodes of similar pain prior to his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he took 3 nitroglycerin tablets sublingually over the past 1 hour, which he states has partially relieved his pain. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.

The patient denies any history of recent surgery, head trauma, recent stroke, abnormal bleeding such as blood in urine or stool or nosebleed.

REVIEW OF SYSTEMS: All other systems reviewed & are negative.

PAST MEDICAL HISTORY: Diabetes mellitus type II, hypertension, coronary artery disease, atrial fibrillation, status post PTCA in 1995 by Dr. ABC.

SOCIAL HISTORY: Denies alcohol or drugs. Smokes 2 packs of cigarettes per day. Works as a banker.

FAMILY HISTORY: Positive for coronary artery disease (father & brother).''',


    '''HISTORY OF PRESENT ILLNESS: This 57-year-old black female complains of having pain and discomfort in the left upper arm, especially when she walks and after heavy meals. This lasts anywhere from a few hours and is not associated with shortness of breath, palpitations, dizziness, or syncope. Patient does not get any chest pain or choking in the neck or pain in the back. Patient denies history of hypertension, diabetes mellitus, enlarged heart, heart murmur, history suggestive of previous myocardial infarction, or acute rheumatic polyarthritis during childhood. Her exercise tolerance is one to two blocks for shortness of breath and easy fatigability.

MEDICATIONS: Patient does not take any specific medications.

PAST HISTORY: The patient underwent hysterectomy in 1986.

FAMILY HISTORY: The patient is married, has four children who are doing fine. Family history is positive for hypertension, congestive heart failure, obesity, cancer, and cerebrovascular accident.

SOCIAL HISTORY: The patient smokes one pack of cigarettes per day and takes drinks on social occasions.'''
]


sample_text = """In short, the patient is a 55-year-old gentleman with long-standing morbid obesity, resistant to nonsurgical methods of weight loss with BMI of 69.7 with comorbidities of hypertension, atrial fibrillation, hyperlipidemia, possible sleep apnea, and also osteoarthritis of the lower extremities. He is also an ex-smoker. He is currently smoking and he is planning to quit and at least he should do this six to eight days before for multiple reasons including decreasing the DVT, PE rates and marginal ulcer problems after surgery, which will be discussed later on."""

### Output Levels

The NER model produces predictions at two distinct output levels:
- document
- chunk

`Note`: By default, the output level is chunk.

#### Output Level: Chunk

  **Input format**:
  
  
  {"**text**": "Input Text from which biology and genetics terms are to be extracted."}

In [8]:
# Example: 1
data_dicts = [
    {
        "text": docs,
        "output_level": "chunk"
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)


,document,ner_chunk,begin,end,ner_label,confidence
0,"HISTORY OF PRESENT ILLNESS: The patient is a 40-year-old white male who presents with a chief complaint of ""chest pain"".\n\nThe patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admits prior episodes of similar pain prior to his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he took 3 nitroglycerin tablets sublingually over the past 1 hour, which he states has partially relieved his pain. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.\n\nThe patient denies any history of recent surgery, head trauma, recent stroke, abnormal bleeding such as blood in urine or stool or nosebleed.\n\nREVIEW OF SYSTEMS: All other systems reviewed & are negative.\n\nPAST MEDICAL HISTORY: Diabetes mellitus type II, hypertension, coronary artery disease, atrial fibrillation, status post PTCA in 1995 by Dr. ABC.\n\nSOCIAL HISTORY: Denies alcohol or drugs. Smokes 2 packs of cigarettes per day. Works as a banker.\n\nFAMILY HISTORY: Positive for coronary artery disease (father & brother).",diabetic,137,144,DIABETES,0.9992
1,"HISTORY OF PRESENT ILLNESS: The patient is a 40-year-old white male who presents with a chief complaint of ""chest pain"".\n\nThe patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admits prior episodes of similar pain prior to his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he took 3 nitroglycerin tablets sublingually over the past 1 hour, which he states has partially relieved his pain. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.\n\nThe patient denies any history of recent surgery, head trauma, recent stroke, abnormal bleeding such as blood in urine or stool or nosebleed.\n\nREVIEW OF SYSTEMS: All other systems reviewed & are negative.\n\nPAST MEDICAL HISTORY: Diabetes mellitus type II, hypertension, coronary artery disease, atrial fibrillation, status post PTCA in 1995 by Dr. ABC.\n\nSOCIAL HISTORY: Denies alcohol or drugs. Smokes 2 packs of cigarettes per day. Works as a banker.\n\nFAMILY HISTORY: Positive for coronary artery disease (father & brother).",coronary artery disease,173,195,CAD,0.6896667
2,"HISTORY OF PRESENT ILLNESS: The patient is a 40-year-old white male who presents with a chief complaint of ""chest pain"".\n\nThe patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vo

In [9]:
# Example: 2
data_dicts = [
    {
        "text": sample_text,
        "output_level": "chunk"
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,document,ner_chunk,begin,end,ner_label,confidence
0,"In short, the patient is a 55-year-old gentleman with long-standing morbid obesity, resistant to nonsurgical methods of weight loss with BMI of 69.7 with comorbidities of hypertension, atrial fibrillation, hyperlipidemia, possible sleep apnea, and also osteoarthritis of the lower extremities. He is also an ex-smoker. He is currently smoking and he is planning to quit and at least he should do this six to eight days before for multiple reasons including decreasing the DVT, PE rates and marginal ulcer problems after surgery, which will be discussed later on.",morbid obesity,68,81,OBESE,0.717


#### Output Level: Document

  **Input format**:
  
  
  {"**text**": "Input Text from which biology and genetics terms are to be extracted.", "**output_level**":"document"}

In [10]:
# Example: 3
data_dicts = [
    {
        "text": docs,
        "output_level": "document"
    }
]

process_data_and_invoke_realtime_endpoint(data_dicts)


,document,ner_chunk,begin,end,ner_label,confidence
0,"HISTORY OF PRESENT ILLNESS: The patient is a 40-year-old white male who presents with a chief complaint of ""chest pain"".\n\nThe patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admits prior episodes of similar pain prior to his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he took 3 nitroglycerin tablets sublingually over the past 1 hour, which he states has partially relieved his pain. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.\n\nThe patient denies any history of recent surgery, head trauma, recent stroke, abnormal bleeding such as blood in urine or stool or nosebleed.\n\nREVIEW OF SYSTEMS: All other systems reviewed & are negative.\n\nPAST MEDICAL HISTORY: Diabetes mellitus type II, hypertension, coronary artery disease, atrial fibrillation, status post PTCA in 1995 by Dr. ABC.\n\nSOCIAL HISTORY: Denies alcohol or drugs. Smokes 2 packs of cigarettes per day. Works as a banker.\n\nFAMILY HISTORY: Positive for coronary artery disease (father & brother).","[diabetic, coronary artery disease, Diabetes mellitus type II, hypertension, coronary artery disease, 1995, ABC, Smokes 2 packs of cigarettes per day, banker]","[137, 173, 1317, 1344, 1358, 1424, 1436, 1483, 1532]","[144, 195, 1341, 1355, 1380, 1427, 1438, 1518, 1537]","[DIABETES, CAD, DIABETES, HYPERTENSION, CAD, PHI, PHI, SMOKER, PHI]","[0.9992, 0.6896667, 0.71244997, 0.987, 0.89136666, 0.9998, 0.9998, 0.63425714, 0.9779]"
1,"HISTORY OF PRESENT ILLNESS: This 57-year-old black female complains of having pain and discomfort in the left upper arm, especially when she walks and after heavy meals. This lasts anywhere from a few hours and is not associated with shortness of breath, palpitations, dizziness, or syncope. Patient does not get any chest pain or choking in the neck or pain in the back. Patient denies history of hypertension, diabetes mellitus, enlarged heart, heart murmur, history suggestive of previous myocardial infarction, or acute rheumatic polyarthritis during childhood. Her exercise tolerance is one to two blocks for shortness of breath and easy fatigability.\n\nMEDICATIONS: Patient does not take any specific medications.\n\nPAST HISTORY: The patient underwent hysterectomy in 1986.\n\nFAMILY HISTORY: The patient is married, has four children who are doing fine. Family history is positive for hypertension, congestive heart failure, obesity, cancer, and cerebrovascular accident.\n\nSOCIAL HISTORY: The patient smokes one pack of cigarettes per day and takes drinks on social occasions.","[hypertension, diabetes mellitus, 1986]","[398, 412, 772]","[409, 428, 775]","[HYPERTENSION, DIABETES, PHI]","[0.9709, 0.8099, 0.9674]"


### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [11]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [12]:
import os

validation_file_name_1 = "input_1.json"
validation_file_name_2 = "input_2.json"
validation_file_name_3 = "input_3.json"

validation_input_path = f"s3://{s3_bucket}/{model_name}/validation-input-json/batch"
validation_output_path = f"s3://{s3_bucket}/{model_name}/validation-output-json/batch"

input_dir = 'inputs/batch'
output_dir = 'outputs/batch'

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [13]:
import json

def write_and_upload_to_s3(json_input_data, file_name):

    json_data = json.dumps(json_input_data)

    with open(file_name, "w") as f:
        f.write(json_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input-json/batch/{os.path.basename(file_name)}",
        Body=(bytes(json_data.encode("UTF-8"))),
    )

In [ ]:
# Define input JSON data for each validation file

input_json_data = {
    validation_file_name_1: {
        "text": docs, 
        "output_level": "chunk"
    },
    validation_file_name_2: {
        "text": sample_text, 
        "output_level": "chunk"
    },
    validation_file_name_3: {
        "text": docs,
        "output_level": "document"
    }
}

# Write and upload each input JSON data to S3
for file_name, json_data in input_json_data.items():
    write_and_upload_to_s3(json_data, f"{input_dir}/{file_name}")

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    accept="application/json",
)
transformer.transform(validation_input_path, content_type=content_type)
transformer.wait()

In [15]:
from urllib.parse import urlparse

def process_s3_output_and_save(validation_file_name, output_file_name):

    output_file_path = f"{output_dir}/{output_file_name}"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}/{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data["predictions"])
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

#### Output Level: Chunk

In [16]:
#Example: 1

process_s3_output_and_save(validation_file_name_1, "out_1.out")

,document,ner_chunk,begin,end,ner_label,confidence
0,"HISTORY OF PRESENT ILLNESS: The patient is a 40-year-old white male who presents with a chief complaint of ""chest pain"".\n\nThe patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admits prior episodes of similar pain prior to his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he took 3 nitroglycerin tablets sublingually over the past 1 hour, which he states has partially relieved his pain. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.\n\nThe patient denies any history of recent surgery, head trauma, recent stroke, abnormal bleeding such as blood in urine or stool or nosebleed.\n\nREVIEW OF SYSTEMS: All other systems reviewed & are negative.\n\nPAST MEDICAL HISTORY: Diabetes mellitus type II, hypertension, coronary artery disease, atrial fibrillation, status post PTCA in 1995 by Dr. ABC.\n\nSOCIAL HISTORY: Denies alcohol or drugs. Smokes 2 packs of cigarettes per day. Works as a banker.\n\nFAMILY HISTORY: Positive for coronary artery disease (father & brother).",diabetic,137,144,DIABETES,0.9992
1,"HISTORY OF PRESENT ILLNESS: The patient is a 40-year-old white male who presents with a chief complaint of ""chest pain"".\n\nThe patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admits prior episodes of similar pain prior to his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he took 3 nitroglycerin tablets sublingually over the past 1 hour, which he states has partially relieved his pain. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.\n\nThe patient denies any history of recent surgery, head trauma, recent stroke, abnormal bleeding such as blood in urine or stool or nosebleed.\n\nREVIEW OF SYSTEMS: All other systems reviewed & are negative.\n\nPAST MEDICAL HISTORY: Diabetes mellitus type II, hypertension, coronary artery disease, atrial fibrillation, status post PTCA in 1995 by Dr. ABC.\n\nSOCIAL HISTORY: Denies alcohol or drugs. Smokes 2 packs of cigarettes per day. Works as a banker.\n\nFAMILY HISTORY: Positive for coronary artery disease (father & brother).",coronary artery disease,173,195,CAD,0.6896667
2,"HISTORY OF PRESENT ILLNESS: The patient is a 40-year-old white male who presents with a chief complaint of ""chest pain"".\n\nThe patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vo

In [17]:
#Example: 2

process_s3_output_and_save(validation_file_name_2, "out_2.out")

,document,ner_chunk,begin,end,ner_label,confidence
0,"In short, the patient is a 55-year-old gentleman with long-standing morbid obesity, resistant to nonsurgical methods of weight loss with BMI of 69.7 with comorbidities of hypertension, atrial fibrillation, hyperlipidemia, possible sleep apnea, and also osteoarthritis of the lower extremities. He is also an ex-smoker. He is currently smoking and he is planning to quit and at least he should do this six to eight days before for multiple reasons including decreasing the DVT, PE rates and marginal ulcer problems after surgery, which will be discussed later on.",morbid obesity,68,81,OBESE,0.717


#### Output Level: Document

In [18]:
#Example: 3

process_s3_output_and_save(validation_file_name_3, "out_3.out")

,document,ner_chunk,begin,end,ner_label,confidence
0,"HISTORY OF PRESENT ILLNESS: The patient is a 40-year-old white male who presents with a chief complaint of ""chest pain"".\n\nThe patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admits prior episodes of similar pain prior to his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he took 3 nitroglycerin tablets sublingually over the past 1 hour, which he states has partially relieved his pain. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.\n\nThe patient denies any history of recent surgery, head trauma, recent stroke, abnormal bleeding such as blood in urine or stool or nosebleed.\n\nREVIEW OF SYSTEMS: All other systems reviewed & are negative.\n\nPAST MEDICAL HISTORY: Diabetes mellitus type II, hypertension, coronary artery disease, atrial fibrillation, status post PTCA in 1995 by Dr. ABC.\n\nSOCIAL HISTORY: Denies alcohol or drugs. Smokes 2 packs of cigarettes per day. Works as a banker.\n\nFAMILY HISTORY: Positive for coronary artery disease (father & brother).","[diabetic, coronary artery disease, Diabetes mellitus type II, hypertension, coronary artery disease, 1995, ABC, Smokes 2 packs of cigarettes per day, banker]","[137, 173, 1317, 1344, 1358, 1424, 1436, 1483, 1532]","[144, 195, 1341, 1355, 1380, 1427, 1438, 1518, 1537]","[DIABETES, CAD, DIABETES, HYPERTENSION, CAD, PHI, PHI, SMOKER, PHI]","[0.9992, 0.6896667, 0.71244997, 0.987, 0.89136666, 0.9998, 0.9998, 0.63425714, 0.9779]"
1,"HISTORY OF PRESENT ILLNESS: This 57-year-old black female complains of having pain and discomfort in the left upper arm, especially when she walks and after heavy meals. This lasts anywhere from a few hours and is not associated with shortness of breath, palpitations, dizziness, or syncope. Patient does not get any chest pain or choking in the neck or pain in the back. Patient denies history of hypertension, diabetes mellitus, enlarged heart, heart murmur, history suggestive of previous myocardial infarction, or acute rheumatic polyarthritis during childhood. Her exercise tolerance is one to two blocks for shortness of breath and easy fatigability.\n\nMEDICATIONS: Patient does not take any specific medications.\n\nPAST HISTORY: The patient underwent hysterectomy in 1986.\n\nFAMILY HISTORY: The patient is married, has four children who are doing fine. Family history is positive for hypertension, congestive heart failure, obesity, cancer, and cerebrovascular accident.\n\nSOCIAL HISTORY: The patient smokes one pack of cigarettes per day and takes drinks on social occasions.","[hypertension, diabetes mellitus, 1986]","[398, 412, 772]","[409, 428, 775]","[HYPERTENSION, DIABETES, PHI]","[0.9709, 0.8099, 0.9674]"


In [19]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-med-ner-risk-factors-pipeline-2024-02-29-14-08-31-415


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

